In [2]:
import dtlpy as dl
import pandas as pd
from datetime import datetime
import os


if dl.token_expired():
    dl.login()

In [22]:
report = []
project = dl.projects.get(project_id='07c9bc7f-4a4c-42e6-b810-4eceeadcc1f5')

for dataset in project.datasets.list():
    if len(dataset.name.split('-')) >= 4:
        report.append({'dataset_name': dataset.name,
                       'dataset_id': dataset.id})
        
pd.DataFrame(report).to_csv('haim_haim.csv', index=False)



    

In [19]:
dataset.items.list()

PagedEntities(page_offset=0, page_size=1000, filters=<dtlpy.entities.filters.Filters object at 0x0000013F82677FA0>, has_next_page=True, total_pages_count=18, items_count=17577)

In [6]:
project = dl.projects.get(project_id='07c9bc7f-4a4c-42e6-b810-4eceeadcc1f5')

for dataset in project.datasets.list():
    if len(dataset.name.split('-')) >= 4:
        print (dataset.name, len(dataset.name.split('-')))

corn-weeds-disease-nutrition-insects-us 6
sugar-cane-weeds-disease-nutrition-br 6
cotton-weeds-disease-nutrition-insects-us 6
sugar-cane-weeds-disease-nutrition-insects-br 7
sugar-beets-weeds-emergence-us 5
grain-sorghum-weeds-emergence-us 5
soybean-weeds-disease-nutrition-insects-us 6
barley-weeds-disease-nutrition-insects-de 6
winter-rapeseed-weeds-disease-nutrition-insects-de 7
soybean-weeds-disease-nutrition-insects-py 6
sugar-cane-weeds-emergence-br 5
soybean-weeds-emergence-br 4
soybean-weeds-disease-nutrition-insects-br 6
soybean-weeds-emergence-us 4
potato-weeds-emergence-us 4
unseeded-weeds-disease-nutrition-insects-py 6
sugar-cane-weeds-disease-nutrition-us 6
sugar-cane-weeds-br 4
sugar-cane-weeds-disease-nutrition-insects-us 7
sunflower-weeds-emergence-bg 4
corn-weeds-emergence-us 4
sugar-beets-weeds-emergence-ma 5
unseeded-weeds-disease-nutrition-insects-us 6
unseeded-weeds-emergence-us 4
sugar-cane-weeds-emergence-us 5
unseeded-weeds-disease-nutrition-br 5
corn-weeds-disea

In [14]:
item

{'annotation_item_id': 1568411,
 'annotation_task_id': 79265,
 'country': 'US',
 'crop': 'Pasture',
 'images_id': [39826545],
 'item_instructions': ['Weeds (Broad/Grass)', 'Emergence'],
 'item_type': 'default',
 'order_id': 592491,
 'status': 'todo',
 'system': {'channels': 3,
  'encoding': '7bit',
  'exif': {'DateTimeOriginal': '2024-04-16T13:33:14.000Z',
   'ExposureTime': 0.002857142857142857,
   'FNumber': 3.98,
   'ISO': 100,
   'Model': 'M30',
   'Orientation': 1,
   'WhiteBalance': 0},
  'height': 6000,
  'isBinary': True,
  'location': {'altitude': 264.123,
   'latitude': 40.043620555555556,
   'longitude': -86.21398508333334},
  'mimetype': 'image/jpeg',
  'originalname': '1568411.jpg',
  'refs': [{'id': '661ff777d88be651fdd86eaa', 'type': 'task'},
   {'id': '661ff778d88be60b50d86eab', 'type': 'assignment'}],
  'size': 26914816,
  'taskStatusLog': [],
  'thumbnailId': '6620eb9bf82e2cb4f150d5c4',
  'width': 8000},
 'task_instructions': ['Weeds (Broad/Grass)', 'Emergence'],
 'ti

In [2]:
project_golden = dl.projects.get(project_id='8c217b4c-53e3-4f05-9205-76bf88ec0838')
dataset_golden = project_golden.datasets.get(dataset_id='65a388a09d96c8a32c7f8b2c')
dataset_test_qualification = project_golden.datasets.get(dataset_id='65a5a76a5d935284046bfe1c')



project_taggers = dl.projects.get(project_id='0c8c900e-468c-4c77-aaab-1871333f772b')
dataset_taggers = project_taggers.datasets.get(dataset_id='65a2c4f2371e289828389c9f')
dataset_test_qualification = project_taggers.datasets.get(dataset_id='65a7fbbee05a6ddc3f8dabfd')

In [3]:
def filter_out_list_of_threats_in_annot():

    list_ai_labels = ["AI Weed Area Tags.Tagging Grid",
    "AI Threats Tags.Unhealthy: Disease/Nutrient deficiency",
    "AI Weed Tags.Identifiable Broadleaf Weed",
    "AI Weed Tags.Identifiable Grass Weed",
    "AI Weed Tags.Suspected missing weeds"]

    filters = dl.Filters()
    filters.resource = dl.FiltersResource.ANNOTATION
    for ai_threat in list_ai_labels:
        filters.add(field='label', values=ai_threat, operator=dl.FiltersOperations.NOT_EQUAL, method=dl.FiltersMethod.AND )

    return filters

In [4]:
def consensus_done ():
    filters = dl.Filters()
    filters.resource = dl.FiltersResource.ITEM
    filters.add(field='metadata.system.refs.metadata.status', values='consensus_done')
    return filters

In [5]:
def generate_report (dataset_golden, dataset_test_qualification):

    report_golden_test = {}

    #change to [0] in real tests!
    task_emergence = dataset_test_qualification.tasks.list()[0]

    assigments_ids = [assig.id for assig in task_emergence.assignments.list()]


    report_golden_test['golden'] = []
    for folder in dataset_test_qualification.items.list(filters = consensus_done()):
        for item_task in folder:
            item_id = item_task.metadata['user']['ID']


            filters = dl.Filters()
            filters.resource = dl.FiltersResource.ITEM
            filters.add(field='metadata.user.ID', values=item_id)

            item_in_folder = dataset_golden.items.list(filters=filters)[0]
            # print (item_in_folder)
            item_gt = item_in_folder[0]
            image_id = item_gt.metadata['user']['ID']


            filter_annot = filter_out_list_of_threats_in_annot()
            list_threats = {}
            list_threats['image_id'] = image_id
            for annot in item_gt.annotations.list(filters=filter_annot):
                list_threats[annot.label] = list_threats.get(annot.label, 0) + 1

            
            report_golden_test['golden'].append(list_threats)

            
            filters_assign = dl.Filters(use_defaults=False)
            filters_assign.add(field='metadata.user.ID', values=item_id)

            filters_assign.add(field='metadata.system.refs.id', values=assigments_ids, operator=dl.FiltersOperations.IN)
            

            for folder in task_emergence.get_items(filters=filters_assign):
                for item_asg in folder:
                    for pass_round in item_asg.metadata['system']['refs']:
                        if pass_round['type'] == 'assignment':
                            assignment_id = (pass_round['id'])
                    
                            annotator = dataset_test_qualification.assignments.get(assignment_id=assignment_id).annotator
                            if annotator not in report_golden_test:
                                report_golden_test[annotator] = []

                    list_threats = {}
                    list_threats['image_id'] = image_id

                    for annot in item_asg.annotations.list(filters=filter_out_list_of_threats_in_annot()):
                        list_threats[annot.label] = list_threats.get(annot.label, 0) + 1
                    report_golden_test[annotator].append(list_threats)
            

    return report_golden_test
        

In [6]:
def get_item_by_image_id(image_id):
    filters = dl.Filters()
    filters.add(field='metadata.user.ID', values=image_id)
    return filters


In [7]:
project_taggers = dl.projects.get(project_id='0c8c900e-468c-4c77-aaab-1871333f772b')
dataset_taggers = project_taggers.datasets.get(dataset_id='65a2c4f2371e289828389c9f')



dataset_test_qualification = project_taggers.datasets.get(dataset_id='65e22c8e264e004b46e18396')
path = r'G:\My Drive\Taggers Qualification - Reports'
owner = dataset_test_qualification.tasks.list()[0].task_owner
dt_name = dataset_test_qualification.name
category = dataset_taggers.name.split(" - ")[1]
new_path = os.path.join(path,owner,category,dt_name)
new_path

'G:\\My Drive\\Taggers Qualification - Reports\\raghunath.m@indivillage.com\\Emergence\\indivillage_emergence_soybean_2024-03-01_dt'

In [8]:
# report_dict = generate_report(dataset_taggers, dataset_test_qualification)

In [9]:
datasets = ['660d2f24ed9ae86a6231cb2a','660d342fb409c5a6c9ab646d']

for dataset_id in datasets:
    dataset_test_qualification = project_taggers.datasets.get(dataset_id=dataset_id)
    crop = dataset_test_qualification.name.lower().split('_')[2]

    report_dict = generate_report(dataset_taggers, dataset_test_qualification)

    list_threats = [f'Plant Population.{crop}_emergence','Weed.Broadleaf - Unidentified','Weed.Grass - Unidentified']

    path = r'G:\My Drive\Taggers Qualification - Reports'
    owner = dataset_test_qualification.tasks.list()[0].task_owner
    category = dataset_taggers.name.split(" - ")[1]

    # Create the folder if it doesn't exist


    new_path = os.path.join(path,owner,category)
    os.makedirs(new_path, exist_ok=True)


    # Create ExcelWriter object
    with pd.ExcelWriter(f'{new_path}\\{dataset_test_qualification.name}.xlsx', engine='xlsxwriter') as writer:

        for label in list_threats:
            # Create DataFrames for each tester
            dfs = {}
            for tester, tester_data in report_dict.items():
                if tester != 'golden':
                
                    dfs[tester] = pd.DataFrame(tester_data).set_index('image_id')[label]

            # Merge DataFrames on 'image_id', fill NaN with 0, and convert to integers
            result_df = pd.concat(dfs.values(), axis=1, keys=dfs.keys()).fillna(0).astype(int)
            # result_df.to_csv('to_test.csv', index=True)

            variance_per_image = round(result_df.var(axis=1), 2)
            result_df['Variance'] = variance_per_image
            sorted_result_df = result_df.sort_values(by='Variance', ascending=False)

            # Function to get the link for each image_id
            def get_link(image_id):
                item_id = dataset_taggers.items.list(filters=get_item_by_image_id(image_id))[0][0].id
                return f'https://console.dataloop.ai/projects/{project_taggers.id}/datasets/{dataset_taggers.id}/items/{item_id}'

            # Apply the function to create a new column 'link'
            sorted_result_df['link'] = sorted_result_df.index.map(get_link)

            # Reorder the columns to have 'link' after 'image_id'
            sorted_result_df = sorted_result_df[['link'] + sorted_result_df.columns.difference(['link']).tolist()]

            # Calculate 'max', 'min', and 'average' for all taggers dynamically
            sorted_result_df.loc[:, 'max'] = sorted_result_df.iloc[:, 2:].max(axis=1)
            sorted_result_df.loc[:, 'min'] = sorted_result_df.iloc[:, 2:].min(axis=1)
            sorted_result_df.loc[:, 'average'] = round(sorted_result_df.iloc[:, 2:].mean(axis=1),2)

            # Reorder the columns to have 'link', 'Variance', 'max', 'min', 'average', and other columns
            columns_order = ['link', 'Variance', 'max', 'min', 'average'] + sorted_result_df.columns[2:-3].tolist()
            sorted_result_df = sorted_result_df[columns_order]

            # Save each DataFrame in a separate sheet with the label as the sheet name
            sorted_result_df.to_excel(writer, sheet_name=label.split(".")[1], index=True)


Iterate Pages: 100%|██████████| 1/1 [00:58<00:00, 58.48s/it]


In [10]:
project = dl.projects.get(project_id='0c8c900e-468c-4c77-aaab-1871333f772b')
dataset_golden = project.datasets.get(dataset_id='65a2c4f2371e289828389c9f')
dataset_test_qualification = project.datasets.get(dataset_id='65e22c8e264e004b46e18396')

report_golden_test = {}

#change to [0] in real tests!
task_emergence = dataset_test_qualification.tasks.list()[0]

assigments_ids = [assig.id for assig in task_emergence.assignments.list()]


report_golden_test['golden'] = []
for folder in dataset_test_qualification.items.list(filters = consensus_done()):
    for item_task in folder:
        item_id = item_task.metadata['user']['ID']


        filters = dl.Filters()
        filters.resource = dl.FiltersResource.ITEM
        filters.add(field='metadata.user.ID', values=item_id)

        item_in_folder = dataset_golden.items.list(filters=filters)[0]
        # print (item_in_folder)
        item_gt = item_in_folder[0]
        image_id = item_gt.metadata['user']['ID']


        filter_annot = filter_out_list_of_threats_in_annot()
        list_threats = {}
        list_threats['image_id'] = image_id
        for annot in item_gt.annotations.list(filters=filter_annot):
            list_threats[annot.label] = list_threats.get(annot.label, 0) + 1

        
        report_golden_test['golden'].append(list_threats)

        
        filters_assign = dl.Filters(use_defaults=False)
        filters_assign.add(field='metadata.user.ID', values=item_id)

        filters_assign.add(field='metadata.system.refs.id', values=assigments_ids, operator=dl.FiltersOperations.IN)
        

        for folder in task_emergence.get_items(filters=filters_assign):
            for folder in task_emergence.get_items(filters=filters_assign):
                for item_asg in folder:
                    for pass_round in item_asg.metadata['system']['refs']:
                        if pass_round['type'] == 'assignment':
                            assignment_id = (pass_round['id'])
                            
                annotator = dataset_test_qualification.assignments.get(assignment_id=assignment_id).annotator
                if annotator not in report_golden_test:
                    report_golden_test[annotator] = []

                list_threats = {}
                list_threats['image_id'] = image_id

                for annot in item_asg.annotations.list(filters=filter_out_list_of_threats_in_annot()):
                    list_threats[annot.label] = list_threats.get(annot.label, 0) + 1
                report_golden_test[annotator].append(list_threats)
        

report_golden_test
        

Iterate Pages:   0%|          | 0/1 [00:00<?, ?it/s]

Iterate Pages: 100%|██████████| 1/1 [00:02<00:00,  2.04s/it]


Iterate Pages: 100%|██████████| 1/1 [00:00<00:00,  1.52it/s]


Iterate Pages: 100%|██████████| 1/1 [00:00<00:00,  1.88it/s]


Iterate Pages: 100%|██████████| 1/1 [00:00<00:00,  1.05it/s]


Iterate Pages: 100%|██████████| 1/1 [00:00<00:00,  2.06it/s]


Iterate Pages: 100%|██████████| 1/1 [00:00<00:00,  2.14it/s]


Iterate Pages: 100%|██████████| 1/1 [00:00<00:00,  1.88it/s]


Iterate Pages: 100%|██████████| 1/1 [00:00<00:00,  1.59it/s]


Iterate Pages: 100%|██████████| 1/1 [00:00<00:00,  1.01it/s]


Iterate Pages: 100%|██████████| 1/1 [00:00<00:00,  1.75it/s]


Iterate Pages: 100%|██████████| 1/1 [00:00<00:00,  1.93it/s]


Iterate Pages: 100%|██████████| 1/1 [00:00<00:00,  1.74it/s]


Iterate Pages: 100%|██████████| 1/1 [00:00<00:00,  1.97it/s]


Iterate Pages: 100%|██████████| 1/1 [00:00<00:00,  1.64it/s]


Iterate Pages: 100%|██████████| 1/1 [00:00<00:00,  1.88it/s]


I

{'golden': [{'image_id': 874968,
   'Plant Population.soybean_emergence': 21,
   'Weed.Grass - Unidentified': 36},
  {'image_id': 368347,
   'Plant Population.soybean_emergence': 87,
   'Insect.Defoliation': 7,
   'Weed.Grass - Unidentified': 15},
  {'image_id': 880808,
   'Plant Population.soybean_emergence': 50,
   'Insect.Defoliation': 2,
   'Weed.Broadleaf - Unidentified': 39,
   'Weed.Grass - Unidentified': 41},
  {'image_id': 387454,
   'Plant Population.soybean_emergence': 73,
   'Weed.Grass - Unidentified': 6},
  {'image_id': 523838,
   'Weed.Grass - Unidentified': 56,
   'Plant Population.soybean_emergence': 13,
   'Weed.Broadleaf - Unidentified': 3},
  {'image_id': 635967,
   'Weed.Grass - Unidentified': 30,
   'Plant Population.soybean_emergence': 6,
   'Weed.Broadleaf - Unidentified': 24},
  {'image_id': 847629,
   'Weed.Broadleaf - Unidentified': 20,
   'Plant Population.soybean_emergence': 29,
   'Weed.Volunteer corn': 4},
  {'image_id': 977387,
   'Plant Population.soybe